# LangGrpah Chatbot

* Real Pyton - [Build Stateful AI Agents in Python](https://realpython.com/langgraph-python/)

> LangGraph expands LangChain’s capabilities by providing tools to build complex LLM workflows with state, conditional edges, and cycles.


* [docs/tutorials/get-started](https://github.com/langchain-ai/langgraph/tree/main/docs/docs/tutorials/get-started)

> To get acquainted with LangGraph's key concepts and features, complete the following LangGraph basics tutorials series:
> 
> 1. Build a basic chatbot
> 2. Add tools
> 3. Add memory
> 4. Add human-in-the-loop controls
> 5. Customize state
> 6. Time travel

In [1]:
import os
from typing import Annotated, List
from typing_extensions import TypedDict

import openai
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model

from IPython.display import Image, display

In [2]:
"""OpenAI chat model listing and connection test using LangChain.

This script:
1. Lists all chat-compatible OpenAI models.
2. Picks the latest GPT model suitable for chat.
3. Tests the selected model with a simple prompt.

Requires:
    - langchain-openai
    - langchain-core
    - openai >= 1.0.0

Environment:
    OPENAI_API_KEY must be set.
"""
def list_openai_models() -> List[str]:
    """List all chat-completion compatible OpenAI models.

    Returns:
        List of chat-compatible model names.
    """
    models = []
    try:
        response = openai.models.list()
        models = [model for model in response.data]
        return models
    except Exception as error:
        print(f"Failed to list models: {error}")
        return []

def list_openai_chat_models() -> List[str]:
    """List all chat-completion compatible OpenAI models.

    Returns:
        List of chat-compatible model names.
    """
    try:
        response = openai.models.list()
        chat_models = []
        for model in response.data:
            if "chat" in model.id.lower():
                chat_models.append(model.id)
        print("Available Chat Models:")
        return chat_models
    except Exception as error:
        print(f"Failed to list models: {error}")
        return []

def test_openai_connection(model) -> None:
    """Test OpenAI connection using the latest chat-compatible model."""
    # chat_models = list_chat_models()
    if not model:
        print("No chat-compatible models available.")
        return

    print(f"\nTesting model: {model}\n")

    try:
        llm = ChatOpenAI(model=model, temperature=1.0)
        prompt_text = "What is the capital of France? Respond with only the name of the city."
        messages = [HumanMessage(content=prompt_text)]
        response = llm.invoke(messages)

        if response.content.strip().lower() == "paris":
            print("CONNECTION SUCCESSFUL!")
            print(f"Model ({llm.model_name}) responded correctly: {response.content}")
        else:
            print("CONNECTION SUCCEEDED, but response was unexpected.")
            print(f"Model Response: {response.content}")

    except Exception as error:
        print("CONNECTION FAILED!")
        print(f"Error Details: {error}")

# Open AI

In [3]:
MODEL: str = 'gpt-4.1'

In [4]:
path_to_openai_key:str = os.path.expanduser('~/.openai/api_key')
with open(path_to_openai_key, 'r', encoding='utf-8') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

In [5]:
test_openai_connection(MODEL)
llm = init_chat_model(f"openai:{MODEL}")


Testing model: gpt-4.1

CONNECTION SUCCESSFUL!
Model (gpt-4.1) responded correctly: Paris


# Chat Bot

## State Machine Memory
A ```StateGraph``` object defines the application as a graph. First step is to define its ```State``` object. The ```State``` includes the graph's schema and reducer functions that handle state updates. In our example, State is a schema with one key: messages. The reducer function is used to append new messages to the list instead of overwriting it. Keys without a reducer annotation will overwrite previous values.

In [6]:
class State(TypedDict):
    """Chat State (message history)
    """
    messages: Annotated[
        list, 
        add_messages
    ]

## ChatBot Function as a Graph Node

Add a ```chatbot``` function as a node to the graph. Nodes represent units of work and are typically regular functions.

In [7]:
def chatbot(state: State):
    """Chat Bot Function which is the basic pattern for all LangGraph node functions.
    The add_messages function in the State instance will append the LLM's response messages.

    Args:
        state: chat history

    """
    return {"messages": [llm.invoke(state["messages"])]}

# The first argument is the unique node name and The second argument is 
# the function or object that will be called whenever he node is used.


In [8]:
# Builder Pattern (GoF), not Factory pattern as you need to specify how to build step by step.
app_graph_builder = StateGraph(State)

# The first argument is the unique node name. The second argument is 
# the function or object that will be called whenever the node is used.
app_graph_builder.add_node("chatbot", chatbot)

## Add START entry point to the ```chatbot``` node in the graph

In [9]:
app_graph_builder.add_edge(START, "chatbot")

## Add END exit point to from the ```chatbot``` node in the graph

In [10]:
app_graph_builder.add_edge("chatbot", END)

## Compile the Greaph

Creates a ```CompiledGraph```.

In [11]:
graph = app_graph_builder.compile()

### Verify the ChatBot as a Graph

In [12]:
# Using https://mermaid.ink/
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)

Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


# Run Chat Bot

In [13]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except Exception as e:
        print(f"failed due to {e}")   

User:  who is Elon musk?


Assistant: **Elon Musk** is a prominent entrepreneur, engineer, and inventor known for founding and leading several groundbreaking companies. He was born on June 28, 1971, in Pretoria, South Africa. Musk is most famously known as the CEO and lead designer of **SpaceX**, CEO and product architect of **Tesla, Inc.**, owner and CTO of **X Corp.** (formerly known as Twitter), and the founder of companies like **The Boring Company**, **Neuralink**, and **OpenAI** (he was a co-founder but is no longer directly involved).

Here are a few highlights about Elon Musk:

- **SpaceX**: Musk founded SpaceX in 2002 with the goal of reducing space transportation costs and enabling the colonization of Mars. SpaceX has developed rockets like Falcon 1, Falcon 9, Falcon Heavy, and the Starship.
- **Tesla, Inc.**: He became involved with Tesla in 2004, soon after its founding, and helped popularize electric vehicles, battery storage, and solar energy products.
- **Twitter/X**: Musk acquired Twitter in 2022

User:  exit


Goodbye!
